In this note book, I
* replicate some of the simulations in the paers, and
* add some variations of my own.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/facebookresearch/mc/blob/master/notebooks/causal_inference_kang_schafer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [3]:
# install the mc package
!pip install -q git+https://github.com/facebookresearch/mc

In [10]:
#@title imports {form-width: "20%"}
import pandas as pd
import numpy as np
import mc

In [11]:
#@title function mc {form-width: "20%"}
def one_run(
    N=400,  # sample size for main data
    n=100,  # sample size for calibration data
    pi=[0.2, 0.8],  # true membership proportions
    p=[[[0.9, 0.2], [0.1, 0.8]], [[0.9, 0.2], [0.1, 0.8]]],  # miscalssification matrix
    mu=[0.8, 0.4],  # true mean of y
    seed=123,  # seed for random data generation
    verbose=False,  # if true, print details
):
    
    # N = 400; n = 100; pi = [0.2, 0.8];
    # p =[[[0.9, 0.2], [0.1, 0.8]], [[0.9, 0.2], [0.1, 0.8]]]
    # mu = [0.8, 0.4]; seed=123

    np.random.seed(seed)
    pi = np.array(pi)
    p = np.array(p)
    mu = np.array(mu)

    i = np.random.binomial(n=1, p=pi[1], size=N + n)  # true group
    y = np.random.binomial(n=1, p=mu[i])  # y_value depends on true group info i
    j = np.random.binomial(n=1, p=p[y, 1, i])  # observed group

    df = pd.DataFrame({
        "sample": ["P"] * N + ["V"] * n,
        "i": i, "j": j, "y": y, "y_squared": y ** 2,
    })

    # start calculation
    df_P = df.query("sample == 'P'")
    df_V = df.query("sample == 'V'")

    n_jd_P = df_P.groupby("j").size().to_numpy()
    y_sum_jd_P = df_P.groupby("j")["y"].sum().to_numpy()
    n_ji_V = pd.crosstab(df_V["j"], df_V["i"]).to_numpy()
    y_sum_ji_V = df_V.pivot_table(
        index="j", columns="i", values="y", aggfunc=np.sum, fill_value=0).to_numpy()
    y2_sum_ji_V = df_V.pivot_table(
        index="j", columns="i", values="y_squared",
        aggfunc=np.sum, fill_value=0).to_numpy()

    # get estimates
    mom = mc.mc_mom(n_jd_P, y_sum_jd_P, n_ji_V, y_sum_ji_V)
    rmle = mc.mc_rmle(n_jd_P,  y_sum_jd_P, n_ji_V, y_sum_ji_V, y2_sum_ji_V)
    out = pd.concat(
        (pd.DataFrame({"mu": mu}), mom, rmle),
         axis=1
    )
    for col in out.columns:
        if col not in ("mu", "mak_li_var"):
            if any(~out[col].between(0., 1.)):
                out[col] = np.nan
    return out

In [12]:
#@title function simulation {form-width: "20%"}
def simulation(n_reps=1000, verbose=True, *args, **kwargs):
    # n_reps=100; verbose=True; args=[]; kwargs={}
    res = pd.concat([
        one_run(seed=seed + 8101352, *args, **kwargs) for seed in range(n_reps) 
    ])

    pct_bad = res.isna().mean()
    est_cols = [col for col in res.columns if col not in ("mu", "mak_li_var")]
    err = res[est_cols].sub(res["mu"], axis=0)
    bias = err.groupby(level=0).mean()
    mse = err.groupby(level=0).apply(lambda df: (df ** 2).mean())


    estimated_var = res.groupby(level=0)['mak_li_var'].mean().to_numpy()
    empirical_var = res.groupby(level=0)["mak_li"].var().to_numpy()

    right_direction = (
        res[est_cols]
        .diff().loc[1].apply(lambda x: (x[~np.isnan(x)] < 0).mean())
    )

    if verbose:
        print(res.head(2))
        print(f"\nbias: \n {bias}")
        print(f"MSE: \n {mse}")
        print(f"\n\n % with bad estimates:\n {pct_bad}")
        print(f"\n\nestimated mak_li_var: {estimated_var}")
        print(f"\n\nempirical mak_li_var: {empirical_var}")
        print(f"\n\nright_direction:\n {right_direction}")

    return {
        "res":res, "err": err, "bias": bias, "mse": mse,
        "pct_bad": pct_bad, "empirical_var": empirical_var,
        "right_direction": right_direction
    }

# simulation()

## simulations in paper

In [13]:
#@title p's that define simulation setup {form-width: "20%"}
p_a = [[[0.9, 0.2], [0.1, 0.8]], [[0.9, 0.2], [0.1, 0.8]]]
p_b = [[[0.8, 0.3], [0.2, 0.7]], [[0.8, 0.3], [0.2, 0.7]]]
p_c = [[[0.93, 0.23], [0.07, 0.77]], [[0.87, 0.17], [0.13, 0.83]]]
p_d = [[[0.95, 0.25], [0.05, 0.75]], [[0.85, 0.15], [0.15, 0.85]]]

In [14]:
#@title a {form-width: "20%"}
a = simulation(p=p_a)

    mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.8  0.631206    0.913043  0.808730  0.803368  0.897701    0.002838
1  0.4  0.409266    0.376623  0.409266  0.402738  0.382089    0.000856

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.188843    0.001773 -0.022137  0.000729  0.006590
1  0.009426   -0.001499 -0.002380 -0.002108 -0.002691
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.037526    0.008667  0.010077  0.007131  0.006804
1  0.001027    0.002806  0.001253  0.000829  0.000906


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.168
with_y_V      0.056
mak_li        0.001
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00579758 0.00092328]


empirical mak_li_var: [0.00676754 0.00089934]


right_direction:
 naive         1.0
validation    1.0
no_y_V        1.0
with_y_V      1.0
mak_li        1.0
dtype: float64


In [ ]:
#@title b {form-width: "20%"}
b = simulation(p=p_b)

    mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.8  0.559524    0.913043  0.736884  0.738098  0.902809    0.003002
1  0.4  0.435345    0.376623  0.431118  0.419763  0.377003    0.000981

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.240917    0.001773 -0.056290 -0.012511  0.006942
1  0.024067   -0.001499 -0.004639 -0.000197 -0.002929
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.059648    0.008667  0.019458  0.010848  0.007229
1  0.001623    0.002806  0.002616  0.001233  0.001045


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.261
with_y_V      0.129
mak_li        0.000
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00649249 0.00104089]


empirical mak_li_var: [0.00718762 0.00103737]


right_direction:
 naive         0.995000
validation    1.000000
no_y_V        0.986468
with_y_V      0.997704
mak_li        1.000000
dtype: float64


In [ ]:
#@title c {form-width: "20%"}
c = simulation(p=p_c)

    mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.8  0.601399    0.913043  0.759488  0.762384  0.894722     0.00287
1  0.4  0.424125    0.376623  0.424125  0.414624  0.384876     0.00086

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.233363    0.001773 -0.099762 -0.071671  0.008238
1  0.032060   -0.001499  0.021654  0.017080 -0.003009
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.056440    0.008667  0.022008  0.013200  0.006683
1  0.001984    0.002806  0.001718  0.001154  0.000928


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.086
with_y_V      0.013
mak_li        0.000
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00585462 0.00092679]


empirical mak_li_var: [0.00662213 0.0009201 ]


right_direction:
 naive         0.993000
validation    1.000000
no_y_V        0.992341
with_y_V      1.000000
mak_li        1.000000
dtype: float64


In [ ]:
#@title d {form-width: "20%"}
d = simulation(p=p_d)

    mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.8  0.570423    0.913043  0.687472  0.715890  0.897034    0.002901
1  0.4  0.441860    0.376623  0.438562  0.425866  0.380724    0.000891

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.262963    0.001773 -0.162411 -0.128344  0.008026
1  0.047320   -0.001499  0.038830  0.031214 -0.002935
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.071085    0.008667  0.039760  0.024237  0.006604
1  0.003189    0.002806  0.002773  0.001839  0.000949


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.053
with_y_V      0.003
mak_li        0.002
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00588446 0.00093662]


empirical mak_li_var: [0.00654655 0.00094103]


right_direction:
 naive         0.949000
validation    1.000000
no_y_V        0.947202
with_y_V      0.990973
mak_li        1.000000
dtype: float64


## simulations with larger primary data

In [ ]:
#@title a2 {form-width: "20%"}
big_N = 40_000
a2 = simulation(
    N=big_N,
    p=p_a
)

    mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.8  0.602140    0.772727  0.743086  0.769419  0.733881    0.004347
1  0.4  0.409569    0.410256  0.384538  0.387136  0.398576    0.000599

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.188340    0.001687 -0.008752  0.006375  0.006507
1  0.012209    0.000073 -0.001510 -0.000508 -0.000546
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.035491    0.008539  0.005806  0.004405  0.006371
1  0.000158    0.002859  0.000149  0.000106  0.000424


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.132
with_y_V      0.026
mak_li        0.002
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00515063 0.00042678]


empirical mak_li_var: [0.00633484 0.00042419]


right_direction:
 naive         1.000
validation    0.999
no_y_V        1.000
with_y_V      1.000
mak_li        1.000
dtype: float64


In [ ]:
#@title b2 {form-width: "20%"}
b2 = simulation(
    N=big_N,
    p=p_a
)

    mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.8  0.602140    0.772727  0.743086  0.769419  0.733881    0.004347
1  0.4  0.409569    0.410256  0.384538  0.387136  0.398576    0.000599

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.188340    0.001687 -0.008752  0.006375  0.006507
1  0.012209    0.000073 -0.001510 -0.000508 -0.000546
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.035491    0.008539  0.005806  0.004405  0.006371
1  0.000158    0.002859  0.000149  0.000106  0.000424


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.132
with_y_V      0.026
mak_li        0.002
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00515063 0.00042678]


empirical mak_li_var: [0.00633484 0.00042419]


right_direction:
 naive         1.000
validation    0.999
no_y_V        1.000
with_y_V      1.000
mak_li        1.000
dtype: float64


In [ ]:
#@title c2 {form-width: "20%"}
c2 = simulation(
    N=big_N,
    p=p_c
)

    mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.8  0.559833    0.772727  0.653137  0.682077  0.737576    0.004648
1  0.4  0.431248    0.410256  0.408020  0.411666  0.397836    0.000590

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.232656    0.001687 -0.088458 -0.089390  0.007551
1  0.034799    0.000073  0.023539  0.024028 -0.000763
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.054147    0.008539  0.013888  0.011228  0.006413
1  0.001220    0.002859  0.000651  0.000654  0.000433


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.049
with_y_V      0.001
mak_li        0.001
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00523006 0.00042944]


empirical mak_li_var: [0.00636242 0.00043298]


right_direction:
 naive         1.000
validation    0.999
no_y_V        1.000
with_y_V      1.000
mak_li        1.000
dtype: float64


In [ ]:
#@title d2 {form-width: "20%"}
d2 = simulation(
    N=big_N,
    p=p_d
)

    mu    naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.8  0.53006    0.772727  0.600169  0.615841  0.732634    0.004843
1  0.4  0.44668    0.410256  0.431582  0.433228  0.402326    0.000588

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.262058    0.001687 -0.157623 -0.162426  0.007649
1  0.049916    0.000073  0.041537  0.041794 -0.000711
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.068693    0.008539  0.029429  0.028114  0.006437
1  0.002501    0.002859  0.001781  0.001793  0.000442


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.029
with_y_V      0.000
mak_li        0.001
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00525504 0.0004399 ]


empirical mak_li_var: [0.00638439 0.00044206]


right_direction:
 naive         1.00000
validation    0.99900
no_y_V        0.99897
with_y_V      1.00000
mak_li        1.00000
dtype: float64


## Collected results

In [ ]:
#@title 1000 X bias and mse {form-width: "20%"}
setups = np.repeat(("a", "b", "c", "d", "a2", "b2", "c2", "d2"), 2)
setups = np.tile(setups, 2)
multipler = 1000
metrics = np.repeat((f"bia X {multipler}", f"mse X {multipler}"), len(setups)/2)
biases = pd.concat([
    r["bias"] * multipler
    for r in (a,b,c,d,a2,b2,c2,d2)
])

mses = pd.concat([
    r["mse"] * multipler
    for r in (a,b,c,d,a2,b2,c2,d2)
])

all = pd.concat([biases, mses])
all["setup"] = setups
all["metric"] = metrics
all["parameter"] = all.index.map({0: "mu1", 1: "mu2"})
all = (
    all.sort_values(["parameter", "metric", "setup",])
    [["parameter", "metric", "setup", "naive", "validation", "no_y_V", "with_y_V", "mak_li"]]
    .round(2)
)
all

,parameter,metric,setup,naive,validation,no_y_V,with_y_V,mak_li
0,mu1,bia X 1000,a,-188.84,1.77,-22.14,0.73,6.59
0,mu1,bia X 1000,a2,-188.34,1.69,-8.75,6.37,6.51
0,mu1,bia X 1000,b,-240.92,1.77,-56.29,-12.51,6.94
0,mu1,bia X 1000,b2,-188.34,1.69,-8.75,6.37,6.51
0,mu1,bia X 1000,c,-233.36,1.77,-99.76,-71.67,8.24
0,mu1,bia X 1000,c2,-232.66,1.69,-88.46,-89.39,7.55
0,mu1,bia X 1000,d,-262.96,1.77,-162.41,-128.34,8.03
0,mu1,bia X 1000,d2,-262.06,1.69,-157.62,-162.43,7.65
0,mu1,mse X 1000,a,37.53,8.67,10.08,7.13,6.80
0,mu1,mse X 1000,a2,35.49,8.54,5.81,4.40,6.37


## smaller effect size

In [ ]:
s1 = simulation(mu=[0.5, 0.4])

    mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.5  0.475177    0.304348  0.540251  0.483457  0.427374    0.004838
1  0.4  0.393822    0.376623  0.393822  0.390383  0.406062    0.000883

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.045793   -0.007387  0.015691  0.006190 -0.002876
1  0.000500   -0.001499 -0.002932 -0.002538 -0.000977
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.003919    0.012915  0.012271  0.006359  0.008862
1  0.000942    0.002806  0.001111  0.000798  0.000953


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.014
with_y_V      0.000
mak_li        0.000
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00795403 0.00091715]


empirical mak_li_var: [0.00886286 0.00095337]


right_direction:
 naive         0.848000
validation    0.777000
no_y_V        0.846856
with_y_V      0.880000
mak_li        0.813000
dtype: float64


In [ ]:
s2 = simulation(mu=[0.5, 0.45])

     mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.50  0.489362    0.304348  0.525633  0.476018  0.438322    0.004838
1  0.45  0.444015    0.402597  0.444015  0.435732  0.445862    0.000906

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.022652   -0.007387  0.010383  0.004051 -0.003576
1 -0.000623   -0.002122 -0.002394 -0.002234 -0.001000
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.002384    0.012915  0.011883  0.006359  0.008933
1  0.000932    0.002807  0.001099  0.000792  0.000928


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.009
with_y_V      0.000
mak_li        0.000
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00798995 0.00093838]


empirical mak_li_var: [0.00892867 0.00092752]


right_direction:
 naive         0.703000
validation    0.650000
no_y_V        0.700303
with_y_V      0.729000
mak_li        0.662000
dtype: float64


In [ ]:
s3 = simulation(mu=[0.5, 0.48])

     mu     naive  validation    no_y_V  with_y_V    mak_li  mak_li_var
0  0.50  0.524823    0.304348  0.580194  0.516845  0.465693    0.004838
1  0.48  0.455598    0.415584  0.455598  0.447596  0.461428    0.000917

bias: 
       naive  validation    no_y_V  with_y_V    mak_li
0 -0.008822   -0.007387  0.007350  0.002770 -0.004374
1 -0.001386   -0.002156 -0.002081 -0.002087 -0.000980
MSE: 
       naive  validation    no_y_V  with_y_V    mak_li
0  0.001986    0.012915  0.011442  0.006413  0.008909
1  0.000951    0.002807  0.001126  0.000813  0.000945


 % with bad estimates:
 mu            0.000
naive         0.000
validation    0.000
no_y_V        0.007
with_y_V      0.000
mak_li        0.000
mak_li_var    0.000
dtype: float64


estimated mak_li_var: [0.00800407 0.00094458]


empirical mak_li_var: [0.00889835 0.00094476]


right_direction:
 naive         0.608000
validation    0.551000
no_y_V        0.607251
with_y_V      0.597000
mak_li        0.562000
dtype: float64
